In [ ]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
# from xgboost import XGBClassifier
# from sklearn.svm import SVC
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import cross_val_score

from sklearn.metrics import roc_auc_score, f1_score


In [ ]:
def train_test_shape(X_train, X_test, y_train, y_test):
    nl = '\n'
    print(f"X_train: {X_train.shape}{nl} X_test: {X_test.shape}{nl} \
    y_train: {y_train.shape}{nl} y_test: {y_test.shape}")

In [ ]:
class Mushrooms():
  DATA = '/content/mushroom.csv'
  TARGET = 'class'
  
  def __init__(self) -> None:
    pass

  def import_data(self):
    return pd.read_csv(self.DATA)
  
  def preprocessing(self,df):
    for i in df:
      uniq = df[i].unique()
      new = []
      for j in df[i]:
          new.append(np.where(uniq==j)[0][0])

      df[i] = new
    return df

  def split(self, df):
    y = df[self.TARGET]
    X = df.loc[:, df.columns != self.TARGET]
    X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.33, random_state=42)
    return X_train, X_test, y_train, y_test


In [ ]:
a = Mushrooms()
df = a.preprocessing(a.import_data())
X_train, X_test, y_train, y_test = a.split(df)
train_test_shape(X_train, X_test, y_train, y_test)

X_train: (5443, 22)
 X_test: (2681, 22)
     y_train: (5443,)
 y_test: (2681,)


In [ ]:
class MushroomsClassif(Mushrooms):
  
  X_train, X_test, y_train, y_test = a.split(df)
  
#   def aa(self):
#     print(self.X_train)

# dd = MushroomsClassif()
# dd.aa()
  
  def logcl(self):
    clf = LogisticRegression(random_state=24, max_iter = 10).fit(self.X_train, self.y_train)
    return clf

  def cv(self, model):
    scores = cross_val_score(model, self.X_train, self.y_train, cv=5)
    return np.mean(scores)

  def metrics():
    f1 = f1_score(y_test, clf.predict(X_test))
    roc_auc = roc_auc_score(y_test ,clf.predict_proba(X_test)[:, 1])
    nl = '\n'
    return f"F1: {f1}{nl} ROC AUC:{roc_auc} "



/usr/local/lib/python3.8/dist-packages/sklearn/linear_model/_logistic.py:458: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(
/usr/local/lib/python3.8/dist-packages/sklearn/linear_model/_logistic.py:458: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _che

0.9423130030249014

In [ ]:
mc = MushroomsClassif()
model = mc.logcl()
mc.cv(model)

In [ ]:
df = reprocessing(df)

In [ ]:
y = df[TARGET]
X = df.loc[:, df.columns != TARGET]

AttributeError: ignored

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.33, random_state=42)

In [ ]:
def train_test_shape():
  nl = '\n'
  print(f"X_train: {X_train.shape}{nl} X_test: {X_test.shape}{nl} y_train: {y_train.shape}{nl} y_test: {y_test.shape}")


train_test_shape()

X_train: (5443, 22)
 X_test: (2681, 22)
 y_train: (5443,)
 y_test: (2681,)


In [ ]:
clf = LogisticRegression(random_state=24, max_iter = 10).fit(X_train, y_train)

/usr/local/lib/python3.8/dist-packages/sklearn/linear_model/_logistic.py:458: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


In [ ]:
scores = cross_val_score(clf, X_train, y_train, cv=y_test.__len__())
np.mean(scores)

Выходные данные были обрезаны до нескольких последних строк (5000).
/usr/local/lib/python3.8/dist-packages/sklearn/linear_model/_logistic.py:458: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(
/usr/local/lib/python3.8/dist-packages/sklearn/linear_model/_logistic.py:458: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/st

0.9388909610841727

In [ ]:
clf.predict(X_test)

array([1, 0, 0, ..., 1, 0, 1])

In [ ]:
f1_score(y_test, clf.predict(X_test))

0.9358517462580185